<a href="https://colab.research.google.com/github/ImagingDataCommons/github_analytics/blob/main/src/jupyterNotebooks/googleAnalyticsApiGa4Only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Google Analytics 4**

###**Environment Setup**

In [ ]:
%%capture
!pip install google-analytics-data

In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
from google.cloud import bigquery
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange,Dimension,Metric,RunReportRequest
client = bigquery.Client(project='idc-external-025')

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "googleAnalyticsApiServiceAccount.json"

In [ ]:
PROPERTY_DATA = [
    {'property_id': '379011209', 'website': 'discourse'},
    {'property_id': '385143649', 'website': 'learn'},
    {'property_id': '385800898', 'website': 'portal'},
    {'property_id': '361361768', 'website': 'viewer'}
]

In [ ]:
# Get yesterday's date
yesterday = datetime.now() - timedelta(days=1)
yesterday_str = yesterday.strftime('%Y-%m-%d')

# Set both start dates to yesterday's date
ga4StartDate = "2023-06-01"
ga4EndDate = yesterday_str

###**Audience Overview**

In [ ]:
def get_ga4_audience_report(property_id):
    """Queries the Acquisition Report using the Google Analytics Data API v1alpha.

    Args:
        property_id: The GA4 property ID to query.

    Returns:
        The Acquisition Report response.
    """
    # Create a GA4 client
    client = BetaAnalyticsDataClient()

    # Define the request
    request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[Dimension(name="date"),
                    Dimension(name="country")
                    ],
        metrics=[Metric(name="totalUsers"),
                 Metric(name="newUsers"),
                 Metric(name="sessions"),
                 Metric(name="engagedSessions"),
                 Metric(name="screenPageViews"),
                 Metric(name="averageSessionDuration")
                 ],
        date_ranges=[DateRange(start_date=ga4StartDate, end_date=ga4EndDate)]
    )

    # Execute the request and get the response
    response = client.run_report(request)

    return response

In [ ]:
def parse_ga4_audience_report(response, property_id, website):
    """Parses the GA4 Audience Report response into a DataFrame.

    Args:
        response: The GA4 Audience Report response.
        property_id: The GA4 property ID.
        website: The corresponding website name.

    Returns:
        A pandas DataFrame containing the parsed GA4 audience data.
    """
    parsed_data = []

    for row in response.rows:
        dimensions = [dim.value for dim in row.dimension_values]
        metrics = [metric.value for metric in row.metric_values]

        parsed_row = [property_id, website] + dimensions + metrics
        parsed_data.append(parsed_row)

    columns = ['property_id', 'website', 'date', 'country', 'totalUsers', 'newUsers', 'sessions', 'engagedSessions','screenPageViews','averageSessionDuration']

    df = pd.DataFrame(parsed_data, columns=columns)
    return df

# Loop through PROPERTY_DATA
ga4_audience_dfs = []  # Rename the list for clarity

for property_info in PROPERTY_DATA:
    property_id = property_info['property_id']
    website = property_info['website']

    # Call the get_ga4_audience_report function to get the GA4 Audience Report response
    response = get_ga4_audience_report(property_id)

    # Parse the response into a DataFrame
    ga4_audience_df = parse_ga4_audience_report(response, property_id, website)

    # Convert the date column to datetime format
    ga4_audience_df['date'] = pd.to_datetime(ga4_audience_df['date'])

    # Append the parsed DataFrame to the list
    ga4_audience_dfs.append(ga4_audience_df)

# Concatenate all parsed GA4 Audience DataFrames into a single DataFrame
combined_ga4_audience_df = pd.concat(ga4_audience_dfs, ignore_index=True)
# Convert 'sessions' and 'averageSessionDuration' columns to float
combined_ga4_audience_df['sessions'] = combined_ga4_audience_df['sessions'].astype(float)
combined_ga4_audience_df['averageSessionDuration'] = combined_ga4_audience_df['averageSessionDuration'].astype(float)

# Calculate total session duration by multiplying averageSessionDuration by sessions
combined_ga4_audience_df['totalSessionDuration'] = combined_ga4_audience_df['averageSessionDuration'] * combined_ga4_audience_df['sessions']

# Display the combined GA4 Audience DataFrame
combined_ga4_audience_df


,property_id,website,date,country,totalUsers,newUsers,sessions,engagedSessions,screenPageViews,averageSessionDuration,totalSessionDuration
0,379011209,discourse,2023-07-17,United States,32,30,34.0,29,105,142.053330,4829.813213
1,379011209,discourse,2023-07-18,United States,30,27,35.0,26,71,92.814704,3248.514651
2,379011209,discourse,2023-07-13,United States,14,11,15.0,9,22,100.083016,1501.245233
3,379011209,discourse,2023-07-17,Netherlands,13,13,13.0,13,26,0.562061,7.306798
4,379011209,discourse,2023-07-18,Germany,11,9,12.0,12,34,339.316574,4071.798889
...,...,...,...,...,...,...,...,...,...,...,...
3214,361361768,viewer,2023-08-22,Sri Lanka,1,1,1.0,1,1,80.575200,80.575200
3215,361361768,viewer,2023-08-23,Canada,1,0,1.0,0,0,0.000000,0.000000
3216,361361768,viewer,2023-08-23,Egypt,1,1,1.0,0,1,0.000000,0.000000
3217,361361768,viewer,2023-08-23,Macao,1,1,1.0,1,1,17.173277,17.173277


In [ ]:
# Define the metrics columns and their corresponding display names
metrics_columns_ga4 = {
    'totalUsers': 'Total Users',
    'newUsers': 'New Users',
    'sessions': 'Sessions',
    'engagedSessions' : 'engagedSessions',
    'screenPageViews': 'Screen Page Views',
    'totalSessionDuration': 'totalSessionDuration'
}

# Cast the columns to float before aggregation
for col in metrics_columns_ga4.keys():
    combined_ga4_audience_df[col] = combined_ga4_audience_df[col].astype(float)

# Group the DataFrame by 'website' and 'date', and calculate aggregates for metrics columns
grouped_ga4_audience_df = combined_ga4_audience_df.groupby(['website', 'date'])[list(metrics_columns_ga4.keys())].sum().reset_index()

# Create a list to store rows for the summary table
summary_rows_ga4 = []

# Populate the summary rows
for index, row in grouped_ga4_audience_df.iterrows():
    date = row['date']
    for metric_column, display_name in metrics_columns_ga4.items():
        summary_rows_ga4.append({
            'date': date,
            'aggregate': float(row[metric_column]),
            'metric_name': display_name,
            'website': row['website']
        })

# Create the summary table DataFrame using pandas.concat
ga4_audience_summary_table = pd.concat([pd.DataFrame([row]) for row in summary_rows_ga4], ignore_index=True)

# Display the summary table for GA4 Audience
ga4_audience_summary_table


,date,aggregate,metric_name,website
0,2023-06-01,6.000000,Total Users,discourse
1,2023-06-01,5.000000,New Users,discourse
2,2023-06-01,6.000000,Sessions,discourse
3,2023-06-01,6.000000,engagedSessions,discourse
4,2023-06-01,33.000000,Screen Page Views,discourse
...,...,...,...,...
1783,2023-08-22,18.000000,New Users,viewer
1784,2023-08-22,32.000000,Sessions,viewer
1785,2023-08-22,17.000000,engagedSessions,viewer
1786,2023-08-22,94.000000,Screen Page Views,viewer


###**Acquisiton Overview**

In [ ]:
def get_ga4_acquisition_report(property_id):
    """Queries the Acquisition Report using the Google Analytics Data API v1alpha.

    Args:
        property_id: The GA4 property ID to query.

    Returns:
        The Acquisition Report response.
    """
    # Create a GA4 client
    client = BetaAnalyticsDataClient()

    # Define the request
    request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[Dimension(name="date"),
                    Dimension(name="sessionDefaultChannelGroup"),
                    Dimension(name="sessionSource")
                    ],
        metrics=[Metric(name="totalUsers"),
                 Metric(name="newUsers"),
                 Metric(name="sessions"),
                 Metric(name="engagedSessions"),
                 Metric(name="screenPageViews"),
                 Metric(name="averageSessionDuration")
                 ],
        date_ranges=[DateRange(start_date=ga4StartDate, end_date=ga4EndDate)]
    )

    # Execute the request and get the response
    response = client.run_report(request)

    return response


In [ ]:
def parse_ga4_acquisition_report(response, property_id, website):
    """Parses the GA4 Acquisition Report response into a DataFrame.

    Args:
        response: The GA4 Acquisition Report response.
        property_id: The GA4 property ID.
        website: The corresponding website name.

    Returns:
        A pandas DataFrame containing the parsed data.
    """
    parsed_data = []

    for row in response.rows:
        dimensions = [dim.value for dim in row.dimension_values]
        metrics = [metric.value for metric in row.metric_values]

        parsed_row = [property_id, website] + dimensions + metrics
        parsed_data.append(parsed_row)

    columns = ['property_id', 'website', 'date', 'channelGrouping', 'sessionSource',
               'totalUsers', 'newUsers', 'sessions', 'engagedSessions','screenPageViews', 'averageSessionDuration']

    df = pd.DataFrame(parsed_data, columns=columns)
    return df

# Loop through PROPERTY_DATA
ga4_acquisition_dfs = []  # Rename the list for clarity

for property_info in PROPERTY_DATA:
    property_id = property_info['property_id']
    website = property_info['website']

    # Call the get_ga4_acquisition_report function to get the GA4 Acquisition Report response
    response = get_ga4_acquisition_report(property_id)

    # Parse the response into a DataFrame
    ga4_acquisition_df = parse_ga4_acquisition_report(response, property_id, website)

    # Convert the date column to datetime format
    ga4_acquisition_df['date'] = pd.to_datetime(ga4_acquisition_df['date'])

    # Append the parsed DataFrame to the list
    ga4_acquisition_dfs.append(ga4_acquisition_df)

# Concatenate all parsed GA4 Acquisition DataFrames into a single DataFrame
combined_ga4_acquisition_df = pd.concat(ga4_acquisition_dfs, ignore_index=True)

combined_ga4_acquisition_df['sessions'] = combined_ga4_acquisition_df['sessions'].astype(float)
combined_ga4_acquisition_df['averageSessionDuration'] = combined_ga4_acquisition_df['averageSessionDuration'].astype(float)
combined_ga4_acquisition_df['totalSessionDuration'] = combined_ga4_acquisition_df['averageSessionDuration'] * combined_ga4_acquisition_df['sessions']
# Display the combined GA4 Acquisition DataFrame
combined_ga4_acquisition_df



,property_id,website,date,channelGrouping,sessionSource,totalUsers,newUsers,sessions,engagedSessions,screenPageViews,averageSessionDuration,totalSessionDuration
0,379011209,discourse,2023-07-18,Direct,(direct),65,61,76.0,65,181,192.396609,14622.142272
1,379011209,discourse,2023-07-17,Direct,(direct),62,61,63.0,52,204,185.329290,11675.745268
2,379011209,discourse,2023-07-19,Direct,(direct),16,10,20.0,17,40,81.334653,1626.693067
3,379011209,discourse,2023-07-18,Organic Social,linkedin.com,14,13,18.0,17,50,44.168648,795.035670
4,379011209,discourse,2023-07-05,Direct,(direct),12,12,12.0,7,25,103.727579,1244.730942
...,...,...,...,...,...,...,...,...,...,...,...,...
1370,361361768,viewer,2023-08-21,Unassigned,(not set),1,0,1.0,0,1,48.487415,48.487415
1371,361361768,viewer,2023-08-22,Referral,appnovation.looker.com,1,0,1.0,0,1,0.000000,0.000000
1372,361361768,viewer,2023-08-22,Referral,humantumoratlas.org,1,1,1.0,0,1,0.000000,0.000000
1373,361361768,viewer,2023-08-22,Referral,localhost,1,1,1.0,1,13,2351.101396,2351.101396


In [ ]:
# Define the metrics columns and their corresponding display names for GA4 Acquisition
metrics_columns_ga4 = {
    'totalUsers': 'Total Users',
    'newUsers': 'New Users',
    'sessions': 'Sessions',
    'engagedSessions': 'Engaged Sessions',
    'screenPageViews': 'Screen Page Views',
    'totalSessionDuration': 'Total Session Duration'
}

# Create a dictionary to map metrics to KPI types for GA4 Acquisition
kpi_mapping_ga4 = {
    'totalUsers': 'acquisition',
    'newUsers': 'acquisition',
    'sessions': 'acquisition',
    'engagedSessions': 'acquisition',
    'screenPageViews': 'behavior',
    'totalSessionDuration': 'behavior'
}

# Cast the columns to float before aggregation for GA4 Acquisition
for col in metrics_columns_ga4.keys():
    combined_ga4_acquisition_df[col] = combined_ga4_acquisition_df[col].astype(float)

# Group the DataFrame by multiple columns for GA4 Acquisition
grouped_ga4_acquisition_df = combined_ga4_acquisition_df.groupby([
    'website', 'date', 'channelGrouping', 'sessionSource'
])[list(metrics_columns_ga4.keys())].sum().reset_index()

# Create a list to store rows for the summary table for GA4 Acquisition
summary_rows_ga4 = []

# Populate the summary rows for GA4 Acquisition
for index, row in grouped_ga4_acquisition_df.iterrows():
    date = row['date']
    channel_grouping = row['channelGrouping']
    for metric_column, display_name in metrics_columns_ga4.items():
        kpi = kpi_mapping_ga4.get(metric_column, 'unknown')  # Get KPI type based on metric_column for GA4 Acquisition
        summary_rows_ga4.append({
            'website': row['website'],
            'date': date,
            'channelGrouping': channel_grouping,
            'sessionSource': row['sessionSource'],
            'metric_name': display_name,
            'kpi': kpi,
            'aggregates': float(row[metric_column])
        })

# Create the summary table DataFrame for GA4 Acquisition using pandas.concat
ga4_acquisition_summary_table = pd.DataFrame(summary_rows_ga4)

# Display the summary table for GA4 Acquisition
ga4_acquisition_summary_table


,website,date,channelGrouping,sessionSource,metric_name,kpi,aggregates
0,discourse,2023-06-01,Direct,(direct),Total Users,acquisition,2.0
1,discourse,2023-06-01,Direct,(direct),New Users,acquisition,1.0
2,discourse,2023-06-01,Direct,(direct),Sessions,acquisition,2.0
3,discourse,2023-06-01,Direct,(direct),Engaged Sessions,acquisition,2.0
4,discourse,2023-06-01,Direct,(direct),Screen Page Views,behavior,17.0
...,...,...,...,...,...,...,...
8245,viewer,2023-08-22,Referral,notebooks.githubusercontent.com,New Users,acquisition,1.0
8246,viewer,2023-08-22,Referral,notebooks.githubusercontent.com,Sessions,acquisition,1.0
8247,viewer,2023-08-22,Referral,notebooks.githubusercontent.com,Engaged Sessions,acquisition,0.0
8248,viewer,2023-08-22,Referral,notebooks.githubusercontent.com,Screen Page Views,behavior,1.0


###**Behavior Overview**

In [ ]:
def get_ga4_behavior_report(property_id):
    """Queries the Acquisition Report using the Google Analytics Data API v1alpha.

    Args:
        property_id: The GA4 property ID to query.

    Returns:
        The Acquisition Report response.
    """
    # Create a GA4 client
    client = BetaAnalyticsDataClient()

    # Define the request
    request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[Dimension(name="date"),
                    Dimension(name="pagePath"),
                    Dimension(name="pageTitle")
                    ],
        metrics=[Metric(name="screenPageViews"),
                 Metric(name="averageSessionDuration"),
                 Metric(name="sessions"),
                 Metric(name="engagedSessions")
                 ],
        date_ranges=[DateRange(start_date=ga4StartDate, end_date=ga4EndDate)]
    )

    # Execute the request and get the response
    response = client.run_report(request)

    return response

In [ ]:
import pandas as pd

def parse_ga4_behavior_report(response, property_id, website):
    """Parses the GA4 Behavior Report response into a DataFrame.

    Args:
        response: The GA4 Behavior Report response.
        property_id: The GA4 property ID.
        website: The corresponding website name.

    Returns:
        A pandas DataFrame containing the parsed GA4 behavior data.
    """
    parsed_data = []

    for row in response.rows:
        dimensions = [dim.value for dim in row.dimension_values]
        metrics = [metric.value for metric in row.metric_values]

        parsed_row = [property_id, website] + dimensions + metrics
        parsed_data.append(parsed_row)

    columns = ['property_id', 'website', 'date', 'pagePath', 'pageTitle', 'screenPageViews',
               'averageSessionDuration', 'sessions', 'engagedSessions']

    df = pd.DataFrame(parsed_data, columns=columns)
    return df

# Loop through PROPERTY_DATA
ga4_behavior_dfs = []  # Rename the list for clarity

for property_info in PROPERTY_DATA:
    property_id = property_info['property_id']
    website = property_info['website']

    # Call the get_ga4_behavior_report function to get the GA4 Behavior Report response
    response = get_ga4_behavior_report(property_id)

    # Parse the response into a DataFrame
    ga4_behavior_df = parse_ga4_behavior_report(response, property_id, website)

    # Convert the date column to datetime format
    ga4_behavior_df['date'] = pd.to_datetime(ga4_behavior_df['date'])

    # Append the parsed DataFrame to the list
    ga4_behavior_dfs.append(ga4_behavior_df)

# Concatenate all parsed GA4 Behavior DataFrames into a single DataFrame
combined_ga4_behavior_df = pd.concat(ga4_behavior_dfs, ignore_index=True)

combined_ga4_behavior_df['sessions'] = combined_ga4_behavior_df['sessions'].astype(float)
combined_ga4_behavior_df['averageSessionDuration'] = combined_ga4_behavior_df['averageSessionDuration'].astype(float)
combined_ga4_behavior_df['totalSessionDuration'] = combined_ga4_behavior_df['averageSessionDuration'] * combined_ga4_behavior_df['sessions']

# Display the combined GA4 Behavior DataFrame
combined_ga4_behavior_df


,property_id,website,date,pagePath,pageTitle,screenPageViews,averageSessionDuration,sessions,engagedSessions,totalSessionDuration
0,379011209,discourse,2023-07-18,/t/idc-july-2023-release/457,IDC July 2023 release - Announcements - Imagin...,295,156.168239,115.0,100,17959.347463
1,379011209,discourse,2023-07-17,/t/idc-july-2023-release/457,IDC July 2023 release - Announcements - Imagin...,187,158.010922,73.0,61,11534.797329
2,379011209,discourse,2023-07-19,/t/idc-july-2023-release/457,IDC July 2023 release - Announcements - Imagin...,66,97.988890,35.0,29,3429.611160
3,379011209,discourse,2023-07-21,/t/idc-july-2023-release/457,IDC July 2023 release - Announcements - Imagin...,40,147.678820,17.0,12,2510.539946
4,379011209,discourse,2023-07-26,/t/idc-july-2023-release/457,IDC July 2023 release - Announcements - Imagin...,31,227.647914,13.0,12,2959.422888
...,...,...,...,...,...,...,...,...,...,...
5426,361361768,viewer,2023-08-22,/slim/studies/2.25.625628962621893025258161596...,Slim,0,95.669055,1.0,0,95.669055
5427,361361768,viewer,2023-08-22,/slim/studies/2.25.688030958969662765833821389...,Slim,0,6.516440,1.0,0,6.516440
5428,361361768,viewer,2023-08-22,/slim/studies/2.25.814650097954192663612734483...,Slim,0,215.684491,1.0,0,215.684491
5429,361361768,viewer,2023-08-22,/slim/studies/2.25.851599177786126159803453057...,Slim,0,187.640945,1.0,0,187.640945


In [ ]:
# Define the metrics columns and their corresponding display names for GA4 Behavior
metrics_columns_ga4 = {
    'screenPageViews': 'Screen Page Views',
    'totalSessionDuration': 'Total Session Duration',
    'sessions': 'Sessions',
    'engagedSessions': 'Engaged Sessions'
}

# Cast the columns to float before aggregation for GA4 Behavior
for col in metrics_columns_ga4.keys():
    combined_ga4_behavior_df[col] = combined_ga4_behavior_df[col].astype(float)

# Group the DataFrame by 'website', 'date', 'pagePath', and 'pageTitle' for GA4 Behavior
grouped_ga4_behavior_df = combined_ga4_behavior_df.groupby([
    'website', 'date', 'pagePath', 'pageTitle'
])[list(metrics_columns_ga4.keys())].sum().reset_index()

# Create a list to store rows for the summary table for GA4 Behavior
summary_rows_ga4_behavior = []

# Populate the summary rows for GA4 Behavior
for index, row in grouped_ga4_behavior_df.iterrows():
    date = row['date']
    for metric_column, display_name in metrics_columns_ga4.items():
        summary_rows_ga4_behavior.append({
            'website': row['website'],
            'date': date,
            'pagePath': row['pagePath'],
            'pageTitle': row['pageTitle'],
            'metric_name': display_name,
            'aggregates': float(row[metric_column])
        })

# Create the summary table DataFrame for GA4 Behavior using pandas.concat
ga4_behavior_summary_table = pd.DataFrame(summary_rows_ga4_behavior)

# Display the summary table for GA4 Behavior
ga4_behavior_summary_table


,website,date,pagePath,pageTitle,metric_name,aggregates
0,discourse,2023-06-01,/,Imaging Data Commons - Imaging Data Commons forum,Screen Page Views,9.000000
1,discourse,2023-06-01,/,Imaging Data Commons - Imaging Data Commons forum,Total Session Duration,78.647192
2,discourse,2023-06-01,/,Imaging Data Commons - Imaging Data Commons forum,Sessions,3.000000
3,discourse,2023-06-01,/,Imaging Data Commons - Imaging Data Commons forum,Engaged Sessions,3.000000
4,discourse,2023-06-01,/c/announcements/5,Latest Announcements topics - Imaging Data Com...,Screen Page Views,2.000000
...,...,...,...,...,...,...
21719,viewer,2023-08-22,/viewer/1.3.6.1.4.1.9328.50.7.3114042253454634...,OHIF Viewer,Engaged Sessions,0.000000
21720,viewer,2023-08-22,/viewer/2.25.106035919049347205763690043295164...,OHIF Viewer,Screen Page Views,3.000000
21721,viewer,2023-08-22,/viewer/2.25.106035919049347205763690043295164...,OHIF Viewer,Total Session Duration,1102.346321
21722,viewer,2023-08-22,/viewer/2.25.106035919049347205763690043295164...,OHIF Viewer,Sessions,3.000000


###**Pushing dataframes to Bigquery tables**

In [ ]:
from google.colab import auth
PROJECT_ID = 'idc-external-025' # Replace <YOUR_PROJECT_ID> with your project ID
auth.authenticate_user()


In [ ]:
combined_ga4_audience_df['totalUsers']=combined_ga4_audience_df['totalUsers'].astype(int)
combined_ga4_audience_df['newUsers']=combined_ga4_audience_df['newUsers'].astype(int)
combined_ga4_audience_df['sessions']=combined_ga4_audience_df['sessions'].astype(int)
combined_ga4_audience_df['engagedSessions']=combined_ga4_audience_df['engagedSessions'].astype(int)
combined_ga4_audience_df['screenPageViews']=combined_ga4_audience_df['screenPageViews'].astype(int)

ga4_audience_job_config = bigquery.LoadJobConfig(schema=[
                                            bigquery.SchemaField("property_id", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("website", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("date", bigquery.enums.SqlTypeNames.TIMESTAMP),
                                            bigquery.SchemaField("country", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("totalUsers", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("newUsers", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("sessions", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("engagedSessions", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("screenPageViews", bigquery.enums.SqlTypeNames.INT64),
                                            bigquery.SchemaField("averageSessionDuration", bigquery.enums.SqlTypeNames.FLOAT64),
                                            bigquery.SchemaField("totalSessionDuration", bigquery.enums.SqlTypeNames.FLOAT64),
                                            ],
                                            #write_disposition="WRITE_TRUNCATE"
                                                        )

ga4_audience_job = client.load_table_from_dataframe(combined_ga4_audience_df, "idc-external-025.logs.google_analytics_ga4_audience" , job_config=ga4_audience_job_config
                                                    )


In [ ]:
ga4_audience_summary_job_config = bigquery.LoadJobConfig(schema=[
                                            bigquery.SchemaField("date", bigquery.enums.SqlTypeNames.TIMESTAMP),
                                            bigquery.SchemaField("aggregate", bigquery.enums.SqlTypeNames.FLOAT64),
                                            bigquery.SchemaField("metric_name", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("website", bigquery.enums.SqlTypeNames.STRING)
                                            ],
                                            #write_disposition="WRITE_TRUNCATE"
                                                         )

ga4_audience_summary_job = client.load_table_from_dataframe(ga4_audience_summary_table, "idc-external-025.logs.google_analytics_ga4_audience_summary", job_config=ga4_audience_summary_job_config)

In [ ]:
ga4_acquisition_summary_job_config = bigquery.LoadJobConfig(schema=[
                                            bigquery.SchemaField("website", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("date", bigquery.enums.SqlTypeNames.TIMESTAMP),
                                            bigquery.SchemaField("channelGrouping", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("sessionSource", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("metric_name", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("kpi", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("aggregates", bigquery.enums.SqlTypeNames.FLOAT64)
                                            ],
                                            #write_disposition="WRITE_TRUNCATE"
                                            )

ga4_acquisition_df_job = client.load_table_from_dataframe(ga4_acquisition_summary_table, "idc-external-025.logs.google_analytics_ga4_acquisition_summary", job_config=ga4_acquisition_summary_job_config)

In [ ]:
ga4_behavior_summary_job_config = bigquery.LoadJobConfig(schema=[
                                            bigquery.SchemaField("website", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("date", bigquery.enums.SqlTypeNames.TIMESTAMP),
                                            bigquery.SchemaField("pagePath", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("pageTitle", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("metric_name", bigquery.enums.SqlTypeNames.STRING),
                                            bigquery.SchemaField("aggregates", bigquery.enums.SqlTypeNames.FLOAT64)
                                            ],
                                            #write_disposition="WRITE_TRUNCATE"
                                                     )
ga4_behavior_df_job = client.load_table_from_dataframe(ga4_behavior_summary_table, "idc-external-025.logs.google_analytics_ga4_behavior_summary", job_config=ga4_behavior_summary_job_config)